# Recruiter Guidelines Experiments

In [9]:
# %pip install langchain langchain-core langchain-community langchain-openai
# %pip install --force-reinstall typing-extensions==4.5
# %pip install --force-reinstall openai==1.8

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
  Using cached typing_extensions-4.10.0-py3-none-any.whl.metadata (3.0 kB)
Using cached typing_extensions-4.10.0-py3-none-any.whl (33 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
  DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are us

## Load Libraries

In [2]:
import os
import sys
import pandas as pd
sys.path.append('..')
from src.prompt import PROMPTS
from src.evaluation import Evalator
from src.helpers import fix_decision_parser
from src.experiment_runner import run_experiment

from huggingface_hub import login
from datasets import Dataset, DatasetDict

pd.set_option('display.max_colwidth', None)

login(token=os.environ.get("HF_TOKEN"), add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (osxkeychain).
Your token has been saved to /Users/nazardrushchak/.cache/huggingface/token
Login successful


## Load Data

In [2]:
df_uk = pd.read_csv('../data/uk_data_samples.csv')
df_en = pd.read_csv('../data/en_data_samples.csv')

## `gpt-3.5-turbo-0125`: English Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [4]:
chain = PROMPTS['recruiter_guidelines_en'] | llm

In [5]:
file_paths = run_experiment(
    folder_path='../data/recruiter_guidelines',
    chain=chain,
    data=df_en,
    lang='en',
    batch_size=5,
)

In [6]:
file_paths

{'gender': '../data/recruiter_guidelines/en/gender.csv',
 'marital_status': '../data/recruiter_guidelines/en/marital_status.csv',
 'military_status': '../data/recruiter_guidelines/en/military_status.csv',
 'religion': '../data/recruiter_guidelines/en/religion.csv',
 'name': '../data/recruiter_guidelines/en/name.csv',
 'age': '../data/recruiter_guidelines/en/age.csv'}

## Load English Results to HF Datasets

In [2]:
FILES_PATHS = {
    'gender': '../data/recruiter_guidelines/en/gender.csv',
    'marital_status': '../data/recruiter_guidelines/en/marital_status.csv',
    'military_status': '../data/recruiter_guidelines/en/military_status.csv',
    'religion': '../data/recruiter_guidelines/en/religion.csv',
    'name': '../data/recruiter_guidelines/en/name.csv',
    'age': '../data/recruiter_guidelines/en/age.csv'
    }

# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():
    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})
    df_part['decision'] = df_part['decision'].map(lambda x: x.lower())
    df_part = df_part.groupby(by=['candidate_id', 'job_id','CV','Job Description', 'Job Position', 'lang', 'protected_group', 'protected_attr', 'group_id']).agg({
        "decision": "first",
        "feedback": "first",
        "raw_ai_decision": "first",
    }).reset_index()
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.28k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en/commit/a09d2b76abd4c9c581cd21ca690fdf1c50f88a81', commit_message='Upload dataset', commit_description='', oid='a09d2b76abd4c9c581cd21ca690fdf1c50f88a81', pr_url=None, pr_revision=None, pr_num=None)

In [3]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [4]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-en", "recruiter_guidelines")

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Generating gender split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [5]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,recruiter_guidelines,gender,en,0.7851,0.9694,1.0,"{'Agender': 0.6067, 'Androgynous': 0.6067, 'Bigender': 0.5667, 'Cisgender': 0.6044, 'Demiboy': 0.5889, 'Demigender': 0.5333, 'Demigirl': 0.5911, 'Female': 0.5956, 'Gender Nonconforming': 0.5644, 'Genderfluid': 0.5889, 'Genderqueer': 0.5711, 'Intersex': 0.5556, 'Male': 0.5356, 'Neutrois': 0.5933, 'Non-Binary': 0.64, 'Pangender': 0.5289, 'Queer': 0.5711, 'Third Gender': 0.5222, 'Transgender': 0.5867, 'Two-Spirit': 0.5356}","{'Agender': 0.0422, 'Androgynous': 0.0467, 'Bigender': 0.0422, 'Cisgender': 0.04, 'Demiboy': 0.0333, 'Demigender': 0.0711, 'Demigirl': 0.0311, 'Female': 0.04, 'Gender Nonconforming': 0.0489, 'Genderfluid': 0.0244, 'Genderqueer': 0.0378, 'Intersex': 0.0444, 'Male': 0.06, 'Neutrois': 0.0244, 'Non-Binary': 0.0622, 'Pangender': 0.0756, 'Queer': 0.0289, 'Third Gender': 0.0778, 'Transgender': 0.0444, 'Two-Spirit': 0.0689}"
1,recruiter_guidelines,marital_status,en,0.8167,0.9704,1.0,"{'Civil union': 0.5422, 'Divorced (Divorced)': 0.5711, 'Married (Husband/Wife)': 0.56, 'Unmarried (Single)': 0.5644, 'Widower (Widow)': 0.5444}","{'Civil union': 0.0467, 'Divorced (Divorced)': 0.0444, 'Married (Husband/Wife)': 0.0333, 'Unmarried (Single)': 0.0422, 'Widower (Widow)': 0.0444}"
2,recruiter_guidelines,military_status,en,0.7804,0.9546,1.0,"{'Civilian': 0.5933, 'Military retiree': 0.5222, 'Participant in combat actions': 0.3244, 'Reservist': 0.5622, 'War veteran': 0.5222}","{'Civilian': 0.0756, 'Military retiree': 0.0356, 'Participant in combat actions': 0.2111, 'Reservist': 0.0533, 'War veteran': 0.0533}"
3,recruiter_guidelines,religion,en,0.7772,0.9677,1.0,"{'atheist': 0.5978, 'buddhist': 0.4622, 'christian': 0.5533, 'hindu': 0.4956, 'jain': 0.4911, 'jew': 0.5044, 'muslim': 0.4756, 'sikh': 0.4889, 'zoroastrian': 0.4533}","{'atheist': 0.1067, 'buddhist': 0.0644, 'christian': 0.0711, 'hindu': 0.0489, 'jain': 0.04, 'jew': 0.0311, 'muslim': 0.0467, 'sikh': 0.0467, 'zoroastrian': 0.0467}"
4,recruiter_guidelines,name,en,0.7838,0.9061,1.0,"{'Aaron': 0.5622, 'Amartol': 0.5756, 'Emma': 0.6, 'Khulian': 0.5644, 'Liusia': 0.5733, 'Liusiia': 0.5933, 'Romchyk': 0.5667, 'Siu': 0.5578, 'Tyhran': 0.5711, 'Zenoviia': 0.5733}","{'Aaron': 0.0444, 'Amartol': 0.0267, 'Emma': 0.0333, 'Khulian': 0.0289, 'Liusia': 0.0378, 'Liusiia': 0.0356, 'Romchyk': 0.0311, 'Siu': 0.0356, 'Tyhran': 0.0444, 'Zenoviia': 0.02}"
5,recruiter_guidelines,age,en,0.7833,0.9457,1.0,"{'20': 0.4889, '30': 0.5711, '40': 0.4533, '50': 0.3133, '60': 0.1844, '70': 0.0689}","{'20': 0.1822, '30': 0.2644, '40': 0.1467, '50': 0.0111, '60': 0.1222, '70': 0.2378}"


In [6]:
EVALUATION_FILE  = '../data/evaluation_results_en.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_en.csv


## `gpt-3.5-turbo-0125`: Ukrainian Language experiment

In [3]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model_name="gpt-3.5-turbo-0125", model_kwargs={"seed": 42, "top_p": 0.0}, temperature=0.0)

In [4]:
chain = PROMPTS['recruiter_guidelines_uk'] | llm

In [5]:
file_paths = run_experiment(
    folder_path='../data/recruiter_guidelines',
    chain=chain,
    data=df_uk,
    lang='uk',
    batch_size=25,
)

In [6]:
file_paths

{'gender': '../data/recruiter_guidelines/uk/gender.csv',
 'marital_status': '../data/recruiter_guidelines/uk/marital_status.csv',
 'military_status': '../data/recruiter_guidelines/uk/military_status.csv',
 'religion': '../data/recruiter_guidelines/uk/religion.csv',
 'name': '../data/recruiter_guidelines/uk/name.csv',
 'age': '../data/recruiter_guidelines/uk/age.csv'}

## Load Ukrainian Results to HF Datasets

In [3]:
FILES_PATHS = {
    'gender': '../data/recruiter_guidelines/uk/gender.csv',
    'marital_status': '../data/recruiter_guidelines/uk/marital_status.csv',
    'military_status': '../data/recruiter_guidelines/uk/military_status.csv',
    'religion': '../data/recruiter_guidelines/uk/religion.csv',
    'name': '../data/recruiter_guidelines/uk/name.csv',
    'age': '../data/recruiter_guidelines/uk/age.csv'
    }

def ukr_decision_fixing(x: str) -> str:
    if not x:
        return x
    for word in x.split():
        for part in ['найн', 'наїн', 'наєн', 'прий', 'accept', 'hire']:
            if word.lower().startswith(part):
                return 'hire'
        for part in ['відхил', 'reject', 'відмов', 'вибачте']:
            if word.lower().startswith(part):
                return 'reject'
    return x


# load data and push to huggingface
dataset_dict = {}
for key_name, file_path in FILES_PATHS.items():

    df_part = pd.read_csv(file_path, dtype={'protected_attr': str})

    df_part = fix_decision_parser(df_part)
    df_part['decision'] = df_part['decision'].map(ukr_decision_fixing)
    df_part = df_part.groupby(by=['candidate_id', 'job_id','CV','Job Description', 'Job Position', 'lang', 'protected_group', 'protected_attr', 'group_id']).agg({
        "decision": "first",
        "feedback": "first",
        "raw_ai_decision": "first",
    }).reset_index()
    dataset_dict[key_name] = Dataset.from_pandas(df_part)

DatasetDict(dataset_dict).push_to_hub('Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk', private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/9 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Creating parquet from Arrow format:   0%|          | 0/5 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/3 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/datasets/Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk/commit/8f023a5c178d8cf48f195438144e10b3293a7fcc', commit_message='Upload dataset', commit_description='', oid='8f023a5c178d8cf48f195438144e10b3293a7fcc', pr_url=None, pr_revision=None, pr_num=None)

In [4]:
# # load data from huggingface
# from datasets import load_dataset
# dataset = load_dataset("Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk", split="gender")
# dataset.to_pandas().head()

## Evaluation Scores

In [5]:
evaluator = Evalator("intfloat/multilingual-e5-large", "Stereotypes-in-LLMs/hiring-analyses-recruiter_guidelines-uk", "recruiter_guidelines")

/opt/homebrew/lib/python3.9/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


Generating gender split:   0%|          | 0/9000 [00:00<?, ? examples/s]

Generating marital_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating military_status split:   0%|          | 0/2250 [00:00<?, ? examples/s]

Generating religion split:   0%|          | 0/4050 [00:00<?, ? examples/s]

Generating name split:   0%|          | 0/4500 [00:00<?, ? examples/s]

Generating age split:   0%|          | 0/2700 [00:00<?, ? examples/s]

In [6]:
df_report = evaluator.get_report()
df_report

,experiment_name,protected_group,lang,min_feedback_similarity,median_feedback_similarity,max_feedback_similarity,mean_reject_approve_per_attr,mean_bias_per_attr
0,recruiter_guidelines,gender,uk,0.7792,0.9610,1.0,"{'Інтерсекс': 0.18, 'Агендер': 0.1978, 'Андрогінний': 0.2289, 'Бігендер': 0.22, 'Гендерквір': 0.2311, 'Гендерне невідповідність': 0.0844, 'Гендерфлюїд': 0.1778, 'Дводушний (Твоуспірит)': 0.3511, 'Демігендер': 0.2133, 'Демідівчина': 0.2311, 'Деміхлопчик': 0.1822, 'Квір': 0.1356, 'Небінарний': 0.2844, 'Неутроїс': 0.24, 'Пангендер': 0.2267, 'Трансгендер': 0.12, 'Третя стать': 0.1356, 'Цісгендер': 0.2533, 'Чоловік': 0.2222, 'Жінка': 0.2067}","{'Інтерсекс': 0.0756, 'Агендер': 0.08, 'Андрогінний': 0.0933, 'Бігендер': 0.1244, 'Гендерквір': 0.0867, 'Гендерне невідповідність': 0.1222, 'Гендерфлюїд': 0.0733, 'Дводушний (Твоуспірит)': 0.1933, 'Демігендер': 0.1, 'Демідівчина': 0.1133, 'Деміхлопчик': 0.0822, 'Квір': 0.08, 'Небінарний': 0.1622, 'Неутроїс': 0.0956, 'Пангендер': 0.1267, 'Трансгендер': 0.0867, 'Третя стать': 0.0933, 'Цісгендер': 0.1222, 'Чоловік': 0.0956, 'Жінка': 0.0978}"
1,recruiter_guidelines,marital_status,uk,0.8463,0.9607,1.0,"{'Вдовець/Вдова': 0.1711, 'Неодружений/Неодружена': 0.3111, 'Одружений/Одружена': 0.3467, 'Розлучений/Розлучена': 0.1844, 'Цивільний шлюб': 0.3089}","{'Вдовець/Вдова': 0.1, 'Неодружений/Неодружена': 0.1022, 'Одружений/Одружена': 0.1289, 'Розлучений/Розлучена': 0.1133, 'Цивільний шлюб': 0.0911}"
2,recruiter_guidelines,military_status,uk,0.8533,0.9586,1.0,"{'Ветеран війни': 0.0911, 'Військовий пенсіонер': 0.0756, 'Резервіст': 0.1378, 'Учасник бойових дій': 0.08, 'Цивільний': 0.1889}","{'Ветеран війни': 0.0333, 'Військовий пенсіонер': 0.04, 'Резервіст': 0.0711, 'Учасник бойових дій': 0.0311, 'Цивільний': 0.1044}"
3,recruiter_guidelines,religion,uk,0.8348,0.9610,1.0,"{'атеїст': 0.2133, 'буддист': 0.1044, 'джайніст': 0.0911, 'зороастрист': 0.0867, 'мусульманин': 0.0578, 'сикх': 0.1333, 'християнин': 0.1733, 'єврей': 0.1622, 'індуїст': 0.1311}","{'атеїст': 0.1289, 'буддист': 0.0644, 'джайніст': 0.0511, 'зороастрист': 0.0333, 'мусульманин': 0.0489, 'сикх': 0.0578, 'християнин': 0.0933, 'єврей': 0.1044, 'індуїст': 0.0733}"
4,recruiter_guidelines,name,uk,0.8006,0.9541,1.0,"{'Аарон': 0.2311, 'Амартол': 0.26, 'Емма': 0.2533, 'Зеновія': 0.2756, 'Люся': 0.2467, 'Люсія': 0.3089, 'Ромчик': 0.1867, 'Сю': 0.2422, 'Тигран': 0.2556, 'Хуліан': 0.2733}","{'Аарон': 0.0644, 'Амартол': 0.0978, 'Емма': 0.0911, 'Зеновія': 0.1044, 'Люся': 0.0844, 'Люсія': 0.1333, 'Ромчик': 0.0822, 'Сю': 0.0667, 'Тигран': 0.0889, 'Хуліан': 0.1111}"
5,recruiter_guidelines,age,uk,0.8615,0.9575,1.0,"{'20': 0.1756, '30': 0.2333, '40': 0.1, '50': 0.0533, '60': 0.0267, '70': 0.0222}","{'20': 0.1311, '30': 0.1889, '40': 0.0556, '50': 0.0178, '60': 0.0267, '70': 0.0267}"


In [7]:
EVALUATION_FILE  = '../data/evaluation_results_uk.csv'
evaluator.save_report(df_report, EVALUATION_FILE)

Report saved to ../data/evaluation_results_uk.csv
